In [1]:
import cv2
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import rospy

import pyrealsense2
from realsense_depth import *
import numpy as np
from matplotlib import pyplot as plt

import open3d as o3d
from open3d_ros_helper import open3d_ros_helper as orh
from scipy import spatial
from scipy.spatial.transform import Rotation as R
import math
import tf2_ros, tf
import geometry_msgs.msg
import PyKDL

image = None

rospy.init_node("my_pic", anonymous=True)

tfbuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfbuffer)
br = tf2_ros.TransformBroadcaster()
t = geometry_msgs.msg.TransformStamped()

bridge = CvBridge()
loop_rate = rospy.Rate(0.5) # Node cycle rate (in Hz).
np.set_printoptions(suppress=True)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def grab_frame():
    
    frame_color=rospy.wait_for_message('/camera/color/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_color = bridge.imgmsg_to_cv2(frame_color, desired_encoding='rgb8')
    
    frame_depth = rospy.wait_for_message('/camera/depth/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_depth = bridge.imgmsg_to_cv2(frame_depth)
    
    return cv_image_color, cv_image_depth

def visualize_rgbd(rgbd_image):
    print(rgbd_image)
    #o3d.visualization.draw_geometries([rgbd_image])
    
    #intrinsic = o3d.camera.PinholeCameraIntrinsic()
    #intrinsic.intrinsic_matrix =  [[462.1379699707031, 0.0, 320.0], [0.0, 462.1379699707031, 240.0], [0.0, 0.0, 1.0]]
    #intrinsic.intrinsic_matrix =  [[347.99755859375, 0.0, 320.0], [0.0, 347.99755859375, 240.0], [0.0, 0.0, 1.0]]
    #intrinsic.intrinsic_matrix =  [[602.71783447, 0.0, 313.06835938], [0.0, 601.61364746, 230.37461853], [0.0, 0.0, 1.0]]
    
    #w = 640
    #h = 480
    #fx = 602.71783447
    #fy = 601.61364746
    #cx = 313.06835938
    #cy = 230.37461853
    
    #Color frame ROS camera
    #w = 640
    #h = 480
    #fx = 462.1379699707031
    #fy = 462.1379699707031
    #cx = 320.0
    #cy = 240.0
    
    #Depth frame ROS camera
    w = 640
    h = 480
    fx = 347.99755859375
    fy = 347.99755859375
    cx = 320.0
    cy = 240.0    
    
    
    intrinsic = o3d.camera.PinholeCameraIntrinsic(w, h, fx,fy, cx, cy)
    intrinsic.intrinsic_matrix = [[fx, 0, cx], [0, fy, cy], [0, 0, 1]]
    
    cam = o3d.camera.PinholeCameraParameters()
    cam.intrinsic = intrinsic
    
    #cam.extrinsic = np.array([[0., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 0., 1.]])
    #pcd = o3d.geometry.create_point_cloud_from_rgbd_image(rgbd_image, cam.intrinsic, cam.extrinsic)
    
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic)
    #pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,intrinsic)
    
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down.
    
    #o3d.visualization.draw_geometries([pcd])
    
    return pcd
    
    
def tst_dataset(color_frame,depth_frame):
    color_raw = o3d.geometry.Image(np.asarray(color_frame))
    depth_raw = o3d.geometry.Image(np.asarray(depth_frame.astype(np.uint16)) )
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, convert_rgb_to_intensity=False)
    pcd = visualize_rgbd(rgbd_image)
    return pcd


def box_pos(x_coord, y_coord, width, height, centered=0):
    if centered == 0:
        start_point = (x_coord, y_coord) # represents the top left corner of rectangle
        end_point = (x_coord+width-1, y_coord+height-1)  # represents the bottom right corner of rectangle
    elif centered == 1:
        new_x = x_coord - (np.floor(width/2)-1).astype(int)
        new_y = y_coord - (np.floor(height/2)-1).astype(int)
        start_point = (new_x, new_y) 
        end_point = (new_x+width, new_y+height)
    return start_point, end_point


def fetch_transform(tfbuffer,frame1,frame2,quat=0):
    flag = 0
    while flag==0:
        try:
            trans = tfbuffer.lookup_transform(frame1, frame2, rospy.Time(),rospy.Duration(8.0))
            #print (trans)
            trans = trans.transform  #save translation and rotation
            #rot = PyKDL.Rotation.Quaternion(* [ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w] )
            #ypr = [ i  / np.pi * 180 for i in rot.GetEulerZYX() ]
            #print(ypr[2],ypr[1],ypr[0])
            
            rot = R.from_quat([ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w]) #creates rotation matrix
            rpy = rot.as_euler('XYZ',degrees=True) #extrinsic
            break
        except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException) as e:
            #print ("Fail", e)
            continue
    if quat==0:
        return trans.translation.x, trans.translation.y, trans.translation.z, rpy[0],rpy[1],rpy[2] #ypr[2], ypr[1], ypr[0]
    elif quat==1:
        return trans.translation.x, trans.translation.y, trans.translation.z, trans.rotation.x, trans.rotation.y, trans.rotation.z, trans.rotation.w
    

    

def Axis_angle_to_Quat(vector,angle):
    x=vector[0]
    y=vector[1]
    z=vector[2]
    qx = x * np.sin(angle/2)
    qy = y * np.sin(angle/2)
    qz = z * np.sin(angle/2)
    qw = np.cos(angle/2)
    
    return qx,qy,qz,qw

def Axis_angle_to_Euler(vector,angle):
    x=vector[0]
    y=vector[1]
    z=vector[2]
    s=np.sin(angle)
    c=np.cos(angle)
    t=1-c
    
    if ((x*y*t + z*s) > 0.998):  #north pole singularity detected
        
        heading = 2 * np.arctan2(x*np.sin(angle/2), np.cos(angle/2))
        attitude = np.pi/2
        bank = 0
        return heading, attitude, bank
    
    elif ((x*y*t + z*s) < -0.998):
        
        heading = -2 * np.arctan2(x*np.sin(angle/2), np.cos(angle/2))
        attitude = -np.pi/2
        bank = 0
        return heading, attitude, bank
    
    heading = np.arctan2(y * s- x * z * t , 1 - (y*y+ z*z ) * t)
    attitude = np.arcsin(x * y * t + z * s)
    bank = np.arctan2(x * s - y * z * t , 1 - (x*x + z*z) * t)
    
    return bank, heading, attitude

In [3]:
color_frame, depth_frame = grab_frame()
print("color frame:",color_frame.shape, " Depth frame:",depth_frame.shape)

#Check center pixel distance

point = (320, 240)

#color_frame, depth_frame = grab_frame()

distance = depth_frame[point[1], point[0]]
print("Center of image:", distance, "mm")


pt_cloud = tst_dataset(color_frame, depth_frame)

downpcd = pt_cloud.voxel_down_sample(voxel_size=0.001)
#o3d.visualization.draw_geometries([downpcd])

print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.06, max_nn=30)) #radius in meters
print("Align normals towards camera")
downpcd.orient_normals_towards_camera_location(camera_location=[0, 0, 0])

#o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

#v1_normal = np.array(downpcd.normals)
#v1_normal_pts = np.array(downpcd.points)

#v2_camera = np.array(downpcd.normals)
v2_camera_pts = np.array(downpcd.points)



#idx = np.where(abs(v2_camera_pts[:,2]) < (distance/1000)+0.01)[0] #fetch all indexes of values less than distance of center of img 0.3 in column 3 of rec
idx = np.where(abs(v2_camera_pts[:,2]) < (distance/1000)+0.01)[0] #fetch all indexes of values less than distance of center of img 0.3 in column 3 of rec
#idx = np.where(abs(v2_camera_pts[:,2]) < abs(v2_camera_pts[:,2]).max()-0.02)[0] #Remove ground

print(idx)
filtered_Pt_Cloud = downpcd.select_by_index(idx, invert=False)
#o3d.visualization.draw_geometries([filtered_Pt_Cloud], point_show_normal=True)
nor = np.array(filtered_Pt_Cloud.normals)
pts = np.array(filtered_Pt_Cloud.points)

print()
distance,index = spatial.KDTree(pts).query( filtered_Pt_Cloud.get_center() ) #find coordinates 
                                                                             #that are closest to the center 

print(distance)
print(index)
print()
print("Center Coordinates(ground truth):",filtered_Pt_Cloud.get_center())
print("Points Coordinates(estimated center point):",pts[index])
print("Normal Coordinates(Normal of estimated center point):",nor[index])

color frame: (480, 640, 3)  Depth frame: (480, 640)
Center of image: 217 mm
RGBDImage of size 
Color image : 640x480, with 3 channels.
Depth image : 640x480, with 1 channels.
Use numpy.asarray to access buffer data.
Recompute the normal of the downsampled point cloud
Align normals towards camera
[   137    138    139 ... 145988 146012 146013]

0.0006384502406340135
9743

Center Coordinates(ground truth): [-0.00380388  0.0008697  -0.21342582]
Points Coordinates(estimated center point): [-0.00368968  0.00061495 -0.214     ]
Normal Coordinates(Normal of estimated center point): [ 0.70315711 -0.01588428  0.71085706]


In [4]:
def getRotation2(v1):
    if np.all(v1==[0., 0., 1.]): v1 = [0, 0.000001, 0.999999]
    vec_x = [1.,0.,0.] 
    vec_y = [0.,1.,0.]
    vec_z = [0.,0.,1.] #>>>>>>>>>>>>>>>>>>>>> change these to -1 to flip again
    
    vec1 = v1 / np.linalg.norm(v1)

    #vector_x = np.cross(vec1, vec_x)/np.linalg.norm(np.cross(vec1, vec_x))
    angle_x = (math.acos(np.dot(vec1, vec_x)))

    #vector_y = np.cross(vec1, vec_y)/np.linalg.norm(np.cross(vec1, vec_y))
    angle_y = (math.acos(np.dot(vec1, vec_y)))

    vector_z = np.cross(vec1, vec_z)/np.linalg.norm(np.cross(vec1, vec_z))
    angle_z = (math.acos(np.dot(vec1, vec_z)))

    #Rotation = filtered_Pt_Cloud.get_rotation_matrix_from_axis_angle(angle*vector) #alternative Open3D lib.
    Rotation = R.from_rotvec(angle_z*vector_z)
    
    return Rotation, angle_x, angle_y, angle_z

rotat, angle_x, angle_y, angle_z = getRotation2(nor[index])

flip_frame = R.from_euler('xyz', [0., 180, 0.], degrees=True) 
#flip_frame2 = R.from_euler('xyz', [0., 0., 180.], degrees=True) #Use if-else to switch between cases
#rotat = flip_frame*rotat*flip_frame2  #Use if-else to switch between cases
rotat = flip_frame*rotat  #to match correct normal/plane pose (flip later to send to robot)

r = rotat.as_euler("xyz", degrees=True)
r_quat = rotat.as_quat()            #x,y,z,w format

print("X axis and normal angle in degrees",np.degrees(angle_x))
print()
print("Y axis and normal angle in degrees",np.degrees(angle_y))
print()
print("Z axis and normal angle in degrees",np.degrees(angle_z))
print()

print("Rotation wrt camera:","Roll:", np.round(r[0],3), "Pitch:", np.round(r[1],3), "Yaw:", np.round(r[2],3))
#print("In Quaternion form xyzw:",r_quat)
print()
print("Coordinates wrt camera:",pts[index])

t.header.frame_id = "camera_depth_optical_frame"
t.child_frame_id = "plane"

t.header.stamp = rospy.Time.now()
t.transform.translation.x = pts[index][0]
t.transform.translation.y = pts[index][1]
t.transform.translation.z = -pts[index][2] #becaues z in opposite dirn

#r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)

t.transform.rotation.x = r_quat[0]
t.transform.rotation.y = r_quat[1]
t.transform.rotation.z = r_quat[2]
t.transform.rotation.w = r_quat[3]

br.sendTransform(t)

#transform_plane = fetch_transform(tfbuffer,'world', 'plane',quat=0)

#transform_plane = np.round(transform_plane,3)
#print()
#print("Trans XYZ:                            ",transform_plane[0], transform_plane[1], transform_plane[2])
#print("Rot in world coordinate system (deg):",transform_plane[3],transform_plane[4],transform_plane[5])



[ 0.70315711 -0.01588428  0.71085706] [0. 0. 1.]
Z axis-angle angle in degrees 44.69530890918372    axis: [-0.02258418 -0.99974494  0.        ]

[ 0.70315711 -0.01588428  0.71085706] [1. 0. 0.]
X axis-angle angle in degrees 45.319148263269554    axis: [-0.          0.99975044  0.02233967]

[ 0.70315711 -0.01588428  0.71085706] [0. 1. 0.]
Y axis-angle angle in degrees 90.91014038950696    axis: [-0.71094676  0.          0.70324584]

Rotation wrt camera: Roll: -1.28 Pitch: -44.681 Yaw: 0.526

Coordinates wrt camera: [-0.00368968  0.00061495 -0.214     ]


In [26]:
transform_plane = fetch_transform(tfbuffer,'world', 'camera_depth_optical_frame',quat=0)

transform_plane = np.round(transform_plane,3)
print()
print("Trans XYZ:                            ",transform_plane[0], transform_plane[1], transform_plane[2])
print("Rot in world coordinate system (deg):",transform_plane[3],transform_plane[4],transform_plane[5])

135

In [ ]:
#mesh with coordinate
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])
o3d.visualization.draw_geometries([filtered_Pt_Cloud,mesh], point_show_normal=True)

In [ ]:
za = filtered_Pt_Cloud.get_axis_aligned_bounding_box()
za

In [ ]:
za.get_extent()


start_point,end_point = box_pos(320, 240, 10, 10, centered=1)  #x,y,width,height, 0-- top left coord, 1--- center coord

cf = color_frame
for i in range (color_frame.shape[0]):
    for j in range (color_frame.shape[1]):
        if depth_frame[i][j] == 0:
            cf[i][j] = [0 , 0, 0] 

plt.imshow(cf)
plt.show()
window_name = 'Filtered_image'  # Window name in which image is displayed

cv2.rectangle(cf, start_point, end_point, (0, 0, 255), 1)
cv2.imshow(window_name, cf)
cv2.waitKey(0)
cv2.destroyAllWindows()


#color_frame_crop = cf[start_point[1]:end_point[1], start_point[0]:end_point[0]]

color_frame_crop = cf[start_point[1]:end_point[1], start_point[0]:end_point[0]]
depth_frame_crop = depth_frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]

print(depth_frame_crop.shape)
zz=depth_frame_crop.T
print(zz.shape)

#cv2.imshow("cropped", depth_frame_crop)
#cv2.waitKey(0)
#cv2.destroyAllWindows()